In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
import numpy as np
from credentials import *
import pandas as pd

# Create Spotify object
credentials = SpotifyClientCredentials(client_id=Client_ID, client_secret=Client_Secret)
sp = spotipy.Spotify(client_credentials_manager=credentials)

def search_song(title, artist, matches=1):
    """Search a song on Spotify and return its Spotify ID."""
    query = f'artist:{artist} track:{title}'
    song_ids = []
    try:
        results = sp.search(q=query, type='track', limit=matches)
        if matches > 0:
            for i in range(matches):
                song_ids.append(results['tracks']['items'][i]['id'])
        else:
            song_ids.append(results['tracks']['items'][0]['id'])
    except:
        song_ids.append("not found")
        
    return song_ids[0] if song_ids else 'not found'

# Load the data from the two CSV files
not_hot_songs = pd.read_csv('not_hot_100_cleaned.csv')
hot_songs = pd.read_csv('billboard_top_100.csv')

# Add a new column 'source' to each dataframe
not_hot_songs['source'] = 'not_hot'
hot_songs['source'] = 'hot'

# Concatenate the two dataframes
songs = pd.concat([not_hot_songs, hot_songs])


# Create a new column 'id' with default value as 'not found'
songs['id'] = 'not found'

# Split the songs into chunks of 50
chunks = np.array_split(songs, len(songs)//50 + 1)

def get_chunk_ids(chunk):
    song_ids = []
    indices = []
    for index, song in chunk.iterrows():
        print(f"Getting the id of {song['Song']}")
        id = search_song(song['Song'], song['Artist'])
        if id != 'not found':  # If the song was found, add its ID to the list
            song_ids.append(id)
            indices.append(index)
    
    return song_ids, indices

# Fetch song IDs for all chunks
for chunk in chunks:
    song_ids, indices = get_chunk_ids(chunk)
    time.sleep(10)
    # Create a temporary DataFrame and update the 'id' column
    temp_df = pd.DataFrame({'id': song_ids}, index=indices)
    songs.update(temp_df)

# Drop the rows with 'not found' as the song ID
songs = songs[songs['id'] != 'not found']

# Reset index of songs DataFrame
songs.reset_index(drop=True, inplace=True)

def get_audio_features(list_of_songs):
    """Get audio features of a list of songs."""
    audio_features_dict = {}
    chunk_size = 50

    for i in range(0, len(list_of_songs), chunk_size):
        chunk = list_of_songs[i:i+chunk_size]
        audio_features = sp.audio_features(chunk)

        for features in audio_features:
            for key, value in features.items():
                if key not in audio_features_dict:
                    audio_features_dict[key] = [value]
                else:
                    audio_features_dict[key].append(value)
        
        time.sleep(20)  # To prevent rate limiting

    return audio_features_dict

def add_audio_features(df, audio_features_df):
    """Concatenate original dataframe with audio features dataframe."""
    audio_features_df = pd.DataFrame(audio_features_df)
    extended_df = pd.concat([df, audio_features_df], axis=1)
    return extended_df

# Fetch audio features for all songs
audio_features = get_audio_features(songs['id'].tolist())

# Extend the original dataframe with the audio features
extended_songs = add_audio_features(songs, audio_features)

# Save the extended dataframe to a CSV file
extended_songs.to_csv('extended_songs.csv', index=False)

Getting the id of Beggin'
Getting the id of STAY (with Justin Bieber)
Getting the id of good 4 u
Getting the id of Bad Habits
Getting the id of INDUSTRY BABY (feat. Jack Harlow)
Getting the id of MONTERO (Call Me By Your Name)
Getting the id of Kiss Me More (feat. SZA)
Getting the id of Todo De Ti
Getting the id of Yonaguni
Getting the id of I WANNA BE YOUR SLAVE
Getting the id of Levitating (feat. DaBaby)
Getting the id of Qué Más Pues?
Getting the id of Permission to Dance
Getting the id of Peaches (feat. Daniel Caesar & Giveon)
Getting the id of Butter
Getting the id of traitor
Getting the id of deja vu
Getting the id of Save Your Tears (with Ariana Grande) (Remix)
Getting the id of AM Remix
Getting the id of Need To Know
Getting the id of Volando - Remix
Getting the id of Ain't Shit
Getting the id of drivers license
Getting the id of Heat Waves
Getting the id of Blinding Lights
Getting the id of happier
Getting the id of Pepas
Getting the id of Motley Crew
Getting the id of Friday 

Getting the id of Memory
Getting the id of Me Fije
Getting the id of Volta Bebê, Volta Neném
Getting the id of Aloha
Getting the id of Sexo Virtual
Getting the id of Closer
Getting the id of Wonderwall - Remastered
Getting the id of Wasting Time ( feat. Drake )
Getting the id of I Don't Do Drugs (feat. Ariana Grande)
Getting the id of Renegade (feat. Taylor Swift)
Getting the id of JUGGERNAUT (feat. Lil Uzi Vert & Pharrell Williams)
Getting the id of All Eyes On Me
Getting the id of EVERY CHANCE I GET (feat. Lil Baby & Lil Durk)
Getting the id of Dick (feat. Doja Cat)
Getting the id of Solar Power
Getting the id of Your Power
Getting the id of Lieben wir
Getting the id of Whoopty
Getting the id of Batom de Cereja - Ao Vivo
Getting the id of Prisoner (feat. Dua Lipa)
Getting the id of Stereo Hearts (feat. Adam Levine)
Getting the id of Reloj
Getting the id of malibu
Getting the id of Eladio Carrion: Bzrp Music Sessions, Vol. 40
Getting the id of Do I Wanna Know?
Getting the id of LUMBER

Getting the id of White Horse (Taylor’s Version)
Getting the id of Forever & Always (Taylor’s Version)
Getting the id of Breathe (feat. Colbie Caillat) (Taylor’s Version)
Getting the id of Ruff Ryders' Anthem
Getting the id of That’s When (feat. Keith Urban) (Taylor’s Version) (From The Vault)
Getting the id of Tell Me Why (Taylor’s Version)
Getting the id of You’re Not Sorry (Taylor’s Version)
Getting the id of Chica Ideal
Getting the id of Relación - Remix
Getting the id of Don’t You (Taylor’s Version) (From The Vault)
Getting the id of We Were Happy (Taylor’s Version) (From The Vault)
Getting the id of WACHA
Getting the id of Dancing With The Devil
Getting the id of Gone
Getting the id of What Other People Say
Getting the id of Hello (feat. A Boogie Wit da Hoodie)
Getting the id of Hasta Abajo
Getting the id of Take You Dancing
Getting the id of Unstable (feat. The Kid LAROI)
Getting the id of Stuck with U (with Justin Bieber)
Getting the id of Hayloft
Getting the id of Esquema Pref

Getting the id of Santa Baby
Getting the id of It's Beginning to Look a Lot Like Christmas (with Mitchell Ayres & His Orchestra)
Getting the id of Christmas (Baby Please Come Home)
Getting the id of Step Into Christmas
Getting the id of Show Out (with Skepta & Pop Smoke)
Getting the id of Investe Em Mim
Getting the id of Lance Individual
Getting the id of Jingle Bell Rock - Daryl's Version
Getting the id of Thank God It's Christmas - Non-Album Single
Getting the id of Christmas Lights
Getting the id of Have Yourself A Merry Little Christmas
Getting the id of Santa's Coming for Us
Getting the id of My Only Wish (This Year)
Getting the id of Baby, It's Cold Outside (feat. Meghan Trainor)
Getting the id of Like It's Christmas
Getting the id of Christmas Without You
Getting the id of You Make It Feel Like Christmas (feat. Blake Shelton)
Getting the id of Gnat
Getting the id of Joy To The World
Getting the id of Santa Claus Is Coming to Town
Getting the id of Baby It's Cold Outside (duet wi

Getting the id of Rich Nigga Shit (feat. Young Thug)
Getting the id of Many Men
Getting the id of Slidin
Getting the id of Outta Time (feat. Drake)
Getting the id of My Dawg
Getting the id of Snitches & Rats (feat. Young Nudy)
Getting the id of Brand New Draco
Getting the id of Lemonade (Feat. Roddy Ricch & Don Toliver) [Remix]
Getting the id of Your Man
Getting the id of No Opp Left Behind
Getting the id of Lets Link
Getting the id of RIP Luv
Getting the id of Steppin On Niggas
Getting the id of Gimme Love
Getting the id of Ew
Getting the id of Daylight
Getting the id of Epidemic
Getting the id of bloody valentine
Getting the id of MODUS
Getting the id of Tick Tock
Getting the id of Sanctuary
Getting the id of feel something
Getting the id of Tap In
Getting the id of Pretty Boy (feat. Lil Yachty)
Getting the id of Afterthought
Getting the id of +Linda
Getting the id of Over Now (with The Weeknd)
Getting the id of Hate The Other Side (with Marshmello & The Kid Laroi)
Getting the id of 

Getting the id of VACATION
Getting the id of Big Body Benz
Getting the id of Ich weiß nicht mal wie sie heißt
Getting the id of Fútbol & Rumba (feat. Enrique Iglesias)
Getting the id of Bleed
Getting the id of Snow On Tha Bluff
Getting the id of Djomb - Bien ou quoi
Getting the id of In Your Eyes (feat. Alida)
Getting the id of Stupid Love
Getting the id of Shotta Flow 5
Getting the id of IDK You Yet
Getting the id of Piece Of Your Heart
Getting the id of Na Raba Toma Tapão
Getting the id of La Santa
Getting the id of Make You Mine
Getting the id of ROCKSTAR (feat. Roddy Ricch) - BLM REMIX
Getting the id of Make It Rain
Getting the id of M' Manc (con Geolier & Sfera Ebbasta)
Getting the id of Emotions 2.0
Getting the id of P2
Getting the id of Selfish
Getting the id of Surrender
Getting the id of Turks (with Gunna & ft. Travis Scott)
Getting the id of BYE ME FUI
Getting the id of Vete
Getting the id of Like It Is
Getting the id of My Oh My (feat. DaBaby)
Getting the id of Let Me Down S

Getting the id of I Can Show You
Getting the id of I'm Sorry
Getting the id of Black Swan
Getting the id of Bad Child
Getting the id of Adicto (with Anuel AA & Ozuna)
Getting the id of <3
Getting the id of Bad Girls, Good Vibes
Getting the id of Liberdade Provisória - Ao Vivo
Getting the id of POP
Getting the id of Celebration Station
Getting the id of Bigger Than Life
Getting the id of Venetia
Getting the id of You Better Move
Getting the id of Chrome Heart Tags
Getting the id of Bust Me
Getting the id of René
Getting the id of Secure The Bag
Getting the id of Urgency (feat. Syd)
Getting the id of ON (Feat. Sia)
Getting the id of P FKN R
Getting the id of Modern Loneliness
Getting the id of Heatin Up (feat. Gunna)
Getting the id of SKYBOX
Getting the id of Young & Alive
Getting the id of i'm so tired...
Getting the id of Numbers (feat. Roddy Ricch, Gunna and London On Da Track)
Getting the id of Girl
Getting the id of Puesto Pa' Guerrial
Getting the id of Definitivamente
Getting the i

Getting the id of GATTI
Getting the id of Candy
Getting the id of JACKBOYS
Getting the id of Combatchy (feat. MC Rebecca)
Getting the id of Old Town Road
Getting the id of Let Me Know (I Wonder Why Freestyle)
Getting the id of Ne reviens pas
Getting the id of New Rules
Getting the id of Cheirosa - Ao Vivo
Getting the id of Havana (feat. Young Thug)
Getting the id of Surtada - Remix Brega Funk
Getting the id of Lover (Remix) [feat. Shawn Mendes]
Getting the id of Vampire
Getting the id of Last Night
Getting the id of Fast Car
Getting the id of Calm Down
Getting the id of Flowers
Getting the id of All My Life
Getting the id of Cruel Summer
Getting the id of Fukumean
Getting the id of Karma
Getting the id of Favorite Song
Getting the id of Flooded The Face
Getting the id of Kill Bill
Getting the id of Snooze
Getting the id of Creepin'
Getting the id of Ella Baila Sola
Getting the id of Sure Thing
Getting the id of Need A Favor
Getting the id of Anti-Hero
Getting the id of Something In The